In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import pickle
import math
import json, codecs # for saving and restoring history
import os, sys
import graphviz

from sklearn.model_selection import train_test_split

from autokeras.image.image_supervised import ImageClassifier

import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import plot_model
#from keras.preprocessing.image import ImageDataGenerator
#from keras.preprocessing import image
#from keras.applications import VGG16
#from keras.applications.vgg16 import preprocess_input

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


## Colab configuration

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB is True:
    from google.colab import drive
    drive.mount('/content/gdrive')
    os.chdir('/content/gdrive/My Drive/Colab Notebooks')
    if not os.path.isdir('DL_Assignment'):
        os.mkdir('DL_Assignment')
    if not os.path.isdir('DL_Assignment/content'):
        os.mkdir('DL_Assignment/content')
    os.chdir('DL_Assignment/content')
    print(os.getcwd())

## Folder configuration

### Generate folders, if not existent

In [ ]:
if not os.path.isdir('../data'):
    os.mkdir('../data')
    
if not os.path.isdir('../models'):
    os.mkdir('../models')
    
    
if not os.path.isdir('../models/history'):
    os.mkdir('../models/history')
    
if not os.path.isdir('tmp'):
    os.mkdir('tmp')  

## Utilities

In [ ]:
def create_validation_set(X_train, y_train, fraction=0.5, show_class_balance=True):
    """
    splits the training data into a training and validation set
    :param X_train: the features
    :param y_train: the one-hot-encoded labels
    :param fraction: the fraction of the test set
    :return: a tuple (X_train, X_val, y_train, y_val), where the y values are still one-hot-encoded
    """
    # split into train and test
    X_train, X_val, y_train, y_val = train_test_split(
        X_train,
        y_train,
        test_size=fraction,
        shuffle=True,
        random_state=42
    )
    if show_class_balance is True:
        yt = np.argmax(y_train, axis=1)
        yv = np.argmax(y_val, axis=1)
        # Plot distribution
        plt.suptitle('relative distributions of classes in train and validation set')
        plot_label_distribution_bar(yt, loc='left')
        plot_label_distribution_bar(yv, loc='right')
        plt.legend([
            'train ({0} photos)'.format(len(y_train)),
            'test ({0} photos)'.format(len(y_val))
        ])
        plt.show()
    return X_train, X_val, y_train, y_val


def load_train_data(test=False, fraction=1):
    """
    :param test: load test data instead of train data
    :param fraction: load only a fraction of the data
    :return: the pre-processed X and y data as a tuple of tow numpy array of
    shape (sample_size, x, y, channels) and (sample_size, 6), respectively
    """
    X_url = f'../data/deepsat-sat6/X_{"test" if test is True else "train"}_sat6_small.csv'
    y_url = f'../data/deepsat-sat6/y_{"test" if test is True else "train"}_sat6_small.csv'
    # X
    row_number = 324000 if test is False else 81000
    rows = math.ceil(row_number * fraction)
    X_df = pd.read_csv(X_url, header=None, sep=',', nrows=rows)
    # unfold
    X_np = np.array(X_df)
    X_train = X_np.reshape((-1,
                            28,
                            28,
                            4))
    y_df = pd.read_csv(y_url, header=None, sep=',', nrows=rows)
    y_train = np.array(y_df)
    return (X_train, y_train)


def get_label(y):
    """
    returns the colloquial label associated with class y
    :param y: the one-hot encoded label
    :return: the colloquial name of the label
    """
    annotations = pd.read_csv('data/deepsat-sat6/sat6annotations.csv', header=None)
    return annotations[annotations[np.argmax(y) + 1] == 1][0].item()


# helper function, from https://stackoverflow.com/a/54092401/1477035
def saveHist(path,history):

    new_hist = {}
    for key in list(history.history.keys()):
        if type(history.history[key]) == np.ndarray:
            new_hist[key] == history.history[key].tolist()
        elif type(history.history[key]) == list:
           if  type(history.history[key][0]) == np.float64:
               new_hist[key] = list(map(float, history.history[key]))

    print(new_hist)
    with codecs.open(path, 'w', encoding='utf-8') as f:
        json.dump(new_hist, f, separators=(',', ':'), sort_keys=True, indent=4) 

def loadHist(path):
    with codecs.open(path, 'r', encoding='utf-8') as f:
        n = json.loads(f.read())
    return n

## Load data

In [ ]:
(X_train, y_train) = load_train_data(test=False, fraction=0.1)
X_train, X_val, y_train, y_val = create_validation_set(X_train, y_train, fraction=0.2, show_class_balance=False)

### Normalization

In [ ]:
# Normalization over all training images (pixel-wise)
X_mean = np.mean(X_train, axis = 0)
X_std = np.std(X_train, axis = 0)
print(f'Shape of X_mean: {X_mean.shape}')
print(f'Shape of X_std: {X_std.shape}')
X_train = (X_train - X_mean ) / (X_std + 0.0001)
X_val = (X_val - X_mean ) / (X_std + 0.0001)

## Try a simple CNN

### Model architecture

In [ ]:
name = 'cnn1'
model = Sequential()

# hyperparameters
batch_size = 128
nb_classes = 6
nb_epochs = 30
img_rows, img_cols = 28, 28
channels = 4
kernel_size = (3, 3)
input_shape = (img_rows, img_cols, channels)
pool_size = (2, 2)

# CNN
model.add(Convolution2D(32, kernel_size, padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Convolution2D(64, kernel_size,padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Convolution2D(128, kernel_size,padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))

# FC
model.add(Flatten())
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# Compilation
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

### Train model

In [ ]:
if not os.path.isfile(f'../models/{name}.h5'):
    # fit the model
    history = model.fit(X_train, y_train, 
                      batch_size=batch_size, 
                      epochs=nb_epochs,
                      verbose=2, 
                      validation_data=(X_val, y_val))
    # save the model after training
    model.save(f'../models/{name}.h5')
    # save the history
    saveHist(f'../models/history/{name}.json', history)
    history = history.history # extract dict
else:
    # load the history
    history = loadHist(f'../models/history/{name}.json')
    # load the model
    model = load_model(f'../models/{name}.h5')

### Evaluate model on validation set

In [ ]:
# summarize history for accuracy
plt.plot(history['acc'])
plt.plot(history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='lower right')
plt.show()

print('Confusion matrix')
pred = model.predict(X_val)
accuracy = model.evaluate(X_val, y_val)
print(f'validation accuracy according to model.evaluate: {accuracy}')

# convert back from one-hot-encoding
y_val_old = y_val
y_val = np.argmax(y_val, axis=1)
pred = np.argmax(pred, axis=1)

print(f'validation accuracy according to sklearn.accuracy_score: {accuracy_score(y_val, pred)}')
print(classification_report(y_val, pred))

## Try auto-keras

### Convert back from one-hot encoded

In [ ]:
# NOT NECESSARY FOR AUTOKERAS
## flatten everything but the first dimension
#def prepare_data(X, y):
#    return (X.reshape((-1, np.prod(X.shape[1:]))), np.argmax(y, axis=1))
#    
#X_train, y_train = prepare_data(X_train, y_train)
#X_val, y_val = prepare_data(X_val, y_val)
y_val = y_val_old
y_train = np.argmax(y_train, axis=1)
y_val = np.argmax(y_val, axis=1)

### NAS for 3 hours

In [ ]:
# maybe use searcher_args={'trainer_args':{'max_iter_num':int}} here for setting the max number of epochs
autoclf = ImageClassifier(path='../models/autokeras', verbose=True, augment=False, 
                          searcher_args={'trainer_args':{'max_iter_num':nb_epochs}} )
autoclf.fit(X_train, y_train, time_limit=3 * 60 * 60)
autoclf.final_fit(X_train, y_train, X_val, y_val, retrain=True)

### Evaluation on validation set

In [ ]:
accuracy = autoclf.evaluate(X_val, y_val)
print(f'validation accuracy according to model.evaluate: {accuracy}')

best_pred = autoclf.predict(X_val)
print(f'validation accuracy according to sklearn.accuracy_score: {accuracy_score(y_val, best_pred)}')
print(classification_report(y_val, best_pred))

## Evaluate auto-keras best model on test set

### Load and normalize data

In [ ]:
X_test, y_test = load_train_data(test=True, fraction=1)
# Normalization over all test images with training mean and std
X_test = (X_test - X_mean ) / (X_std + 0.0001)
# convert back from one-hot-encoding
y_test = np.argmax(y_test, axis=1)

In [ ]:
best_pred_test = autoclf.predict(X_test)

print(f'test accuracy according to sklearn.accuracy_score: {accuracy_score(y_test, best_pred_test)}')
print(classification_report(y_test, best_pred_test))

### Visualize auto-keras model

In [ ]:
autoclf.export_autokeras_model('../models/autokeras.h5')
autoclf.export_keras_model('../models/autokeras_keras_model.h5')
autoclf.load_searcher().load_best_model().produce_keras_model().save('models/autokeras_keras_model_1.h5')


In [ ]:
model = load_model('../models/autokeras_keras_model.h5')
plot_model(model, to_file='../models/best_autokeras_model.png')